In [20]:
import warnings
warnings.filterwarnings("ignore")

from carla.data.catalog import CsvCatalog, OnlineCatalog
from carla import MLModelCatalog
from carla.recourse_methods import Clue, Wachter
from carla.models.negative_instances import predict_negative_instances
import numpy as np
from pandas import DataFrame
from sklearn.metrics.pairwise import rbf_kernel


import sys
from copy import deepcopy
sys.path.insert(0,'..')
from recourse_util import update_dataset, train_recourse_method, predict, print_scores 

num = 10

In [21]:
dataset = CsvCatalog(
    # file_path='datasets/bimodal_dataset_1.csv',
    file_path='datasets/unimodal_dataset_1.csv',
    # file_path='datasets/unimodal_dataset_2.csv',
    categorical=[],
    continuous=['feature1', 'feature2'],
    immutables=[],
    target='target'
)

In [22]:
dataset = OnlineCatalog('compas')

In [23]:
training_params = {"lr": 0.01, "epochs": 4, "batch_size": 1, "hidden_size": [4]}

model = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=True,
    backend="pytorch"
)

model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)

model1 = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=False,
    backend="pytorch"
)

model1.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)

balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/3
----------
train Loss: 0.3875 Acc: 0.8300

test Loss: 0.3819 Acc: 0.8010

Epoch 1/3
----------
train Loss: 0.3758 Acc: 0.8341

test Loss: 0.3389 Acc: 0.8509

Epoch 2/3
----------
train Loss: 0.3700 Acc: 0.8423

test Loss: 0.3440 Acc: 0.8490

Epoch 3/3
----------
train Loss: 0.3705 Acc: 0.8391

test Loss: 0.3422 Acc: 0.8483

balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/3
----------
train Loss: 0.3912 Acc: 0.8285

test Loss: 0.3615 Acc: 0.8315

Epoch 1/3
----------
train Loss: 0.3726 Acc: 0.8391

test Loss: 0.3410 Acc: 0.8516

Epoch 2/3
----------
train Loss: 0.3693 Acc: 0.8414

test Loss: 0.3511 Acc: 0.8477

Epoch 3/3
----------
train Loss: 0.3687 Acc: 0.8404

test Loss: 0.3429 Acc: 0.8496



In [24]:
factuals = predict_negative_instances(model, dataset._df).sample(num)

In [25]:
hyperparams = {
                "data_name": 'custom',
                "train_vae": True,
                "width": 10,
                "depth": 3,
                "latent_dim": 12,
                "batch_size": 4,
                "epochs": 5,
                "lr": 0.0001,
                "early_stop": 20,
            }

cl = train_recourse_method(dataset, model, 'custom', 'CLUE', hyperparams)

[INFO] 
Net: [utils.py __init__]
[INFO] VAE_gauss_net [fc_gauss_cat.py __init__]
[INFO] Total params: 0.00M [fc_gauss_cat.py create_net]
[INFO] 
Network: [train.py train_VAE]
[INFO] 
Train: [train.py train_VAE]
[INFO] init cost variables: [train.py train_VAE]


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 10])

In [6]:
counterfactuals = cl.get_counterfactuals(factuals)

post = deepcopy(dataset)
update_dataset(post, factuals, counterfactuals)

In [7]:
from scipy.spatial.distance import squareform, pdist
sigma = np.median(pdist(factuals.append(counterfactuals)))

In [8]:
def k(a, b, sigma):
    sq_dist = abs(np.sum((a - b) ** 2, axis=0))
    return np.exp(-(1/sigma) * sq_dist)

In [9]:
def mmd_u(df_a, df_b):
    df_a = df_a.drop(dataset.target, axis=1)
    df_b = df_b.drop(dataset.target, axis=1)
    
    sigma = np.median(pdist(df_a.append(df_b)))
    
    total = 0
    len_a = len(df_a)
    for i in range(len_a):
        for j in range(len(df_b)):
            if i != j:
                h = k(df_a.iloc[i], df_a.iloc[j], sigma) + k(df_b.iloc[i], df_b.iloc[j], sigma)
                - k(df_a.iloc[i], df_b.iloc[j], sigma) - k(df_a.iloc[j], df_b.iloc[i], sigma)
                total += h
                    
    return np.sqrt(total / (len_a * (len_a - 1)))

In [10]:
mmd(dataset.df.iloc[0:2], dataset.df.iloc[0:2])

NameError: name 'mmd' is not defined

In [ ]:
t = abs(np.sum((dataset.df.iloc[1] - dataset.df.iloc[0]) ** 2, axis=0))

In [ ]:
np.exp(-t)

In [ ]:
def mmd(df_a, df_b):
    df_a = df_a.drop(dataset.target, axis=1)
    df_b = df_b.drop(dataset.target, axis=1)
    
    df_c = df_a.append(df_b)
    
    distances = pdist(df_c, 'sqeuclidean')
    
    sigma = np.sqrt(np.median(distances))
    
    total = 0
    len_a = len(df_a)
    len_b = len(df_b)
    len_c = len(df_c)
    
    def get_dist_index(i, j, m):
        return int(m * i + j - ((i + 2) * (i + 1)) / 2)
    
    def k_fun(dist, sigma):
        return np.exp(-(1/sigma) * dist)
    
    for i in range(len_a):
        for j in range(len_b):
            if i != j:
                total += k_fun(distances[get_dist_index(i, j, len_c)], sigma) / (len_a ** 2 - len_a)
                total += k_fun(distances[get_dist_index(i + len_a, j + len_a, len_c)], sigma) / (len_b ** 2 - len_b)
            total += k_fun(distances[get_dist_index(i, j + len_a, len_c)], sigma) / (len_a * len_b)
                    
    return total / (len_a * (len_a - 1))

In [ ]:
a, b = (dataset.df.sample(50), dataset.df.sample(50))

In [ ]:
import timeit

In [ ]:
bef = timeit.default_timer()
for _ in range(1000):
    mmd(a, b)
aft = timeit.default_timer()
aft - bef

In [ ]:
mmd(a, b)

In [ ]:
squareform(pdist(c))

In [ ]:
def disagreement(model_a, model_b, data):
    pred_a = np.where(model_a.predict(data.df), 1, 0)
    pred_b = np.where(model_b.predict(data.df), 1, 0)
    return sum([1 if a != b else 0 for (a, b) in zip(pred_a, pred_b)])/len(data.df)

In [ ]:
disagreement(model, model1, dataset)

In [13]:
def mmd(df_a: DataFrame, df_b: DataFrame, target: str) -> float:
    """
    Computes the Maximum Mean Discrepancy metric using formula from
    Gretton et al. (2012) https://dl.acm.org/doi/10.5555/2188385.2188410
    Uses sklearn.metrics.pairwise.rbf_kernel, it is more stable than the
    manual kernel calculation method.
    :param df_a: DataFrame of the first distribution
    :param df_b: DataFrame of the second distribution
    :param target: str
    :return: float MMD metric for the two DataFrames
    """
    df_a = df_a.loc[df_a[target] == 1].sample(100, replace=True).drop(target, axis=1)
    df_b = df_b.loc[df_b[target] == 1].sample(100, replace=True).drop(target, axis=1)

    len_a = len(df_a)
    len_b = len(df_b)

    df_c = df_a.append(df_b)

    distances = pdist(df_c, 'sqeuclidean')

    sigma = np.sqrt(np.median(distances))

    total = 0

    total += np.sum(rbf_kernel(df_a, gamma=1 / sigma), axis=None) / (len_a ** 2 - len_a)
    total += np.sum(rbf_kernel(df_b, gamma=1 / sigma), axis=None) / (len_b ** 2 - len_b)
    total -= 2 * np.sum(rbf_kernel(df_a, df_b, gamma=1 / sigma), axis=None) / (len_a * len_b)

    return total

In [14]:
a, b = (dataset.df.sample(50), dataset.df.sample(50))
import timeit

In [26]:
bef = timeit.default_timer()
for _ in range(1000):
    mmd(a, b, dataset.target)
aft = timeit.default_timer()
aft - bef

KeyError: 'score'